# Prediction

#### Input
1. Reusable trained model at model_dir which is 'model/linear' or 'model/dnn' that has been created by DNNClassifier_Training.ipynb
1. Target data and maybe label : %restore -r data_label 

#### Output 
1. csv file predictions.csv 


In [1]:
import tensorflow as tf
import numpy as np
import peforth

## Get test data and label

In [2]:
%store -r data_label

X = data_label[0]  # features pandas data-frame
y = data_label[1]  # labels pandas series <class 'pandas.core.series.Series'>
label = list(y)    # Casting

In [3]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=X, y=y, 
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


### Define classifier 所需要的 feature_columns


In [4]:
%run feature_columns_for_DNN.ipynb

<IPython.core.display.Javascript object>

Stored 'feature_columns' (list)


### 祭出 DNNClassifier 

Classifier 無法跨 notebook save-restore 只能當地定義，所以恐怕會有多處定義不一致的狀況發生。
可以引用同一 source code string 的方式解決。

In [5]:

classifier = tf.estimator.DNNClassifier(
    hidden_units=[24, 24, 24],  # 只隨便用了小小三層神經網路兒，對付 WH300 已經很好了 
    feature_columns=feature_columns, 
    n_classes=2, 
    model_dir='model/dnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000202811686A0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [7]:
# 把 predictions 由 0 與 1 的「機率」轉成 0 與 1 
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in preds];

INFO:tensorflow:Restoring parameters from model/dnn\model.ckpt-14000


In [8]:
# 看一看預測出來的 tags 
print(predictions[:100])    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
# 如果有手工 lable 則比對看看
print(label[:100])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# 比較 tag 的個數
print(np.sum(predictions))
print(np.sum(label))

117
0


In [11]:
predictions.__len__()

852

In [12]:
label.__len__()

852

In [13]:
predictions.__len__()

852

In [14]:
predictions[0]==label[0]

True

In [15]:
%pdb off
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(predictions.__len__()): 
    if predictions[i]==int(label[i]) and label[i]==0: categories[0]+=1; continue
    if predictions[i]!=int(label[i]) and label[i]==1: categories[1]+=1; continue
    if predictions[i]!=int(label[i]) and label[i]==0: categories[2]+=1; continue
    if predictions[i]==int(label[i]) and label[i]==1: categories[3]+=1; continue
%f __main__ :> categories ( 沒事的，漏掉的，多的，一致的 ) tib.

Automatic pdb calling has been turned OFF
__main__ :> categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [735, 0, 117, 0] (<class 'list'>)


# 輸出 預測結果 到 predictions.csv 檔
可以用 excel 打開，手動合併到總表裡去，就得到了每件物料的 Tag.

In [17]:
# 輸出 預測結果 到 predictions.csv 檔
bc = list(X['Barcode'])  # Barcode is the id of each item

if bc.__len__() != predictions.__len__() :
    raise Exception("預測出來的結果總數 %i 與給定的資料筆數 %i 不同！" % (predictions.__len__(),bc.__len__()))

predcsv = "Barcode,Tag\n"
for i in range(predictions.__len__()): 
    predcsv += bc[i] + ',' + str(predictions[i]) + '\n';
    
peforth.push(predcsv).dictate('char predictions.csv writeTextFile');  # save the predictions to csv file so you can merge it to excel 